<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [55]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [56]:
import folium
import pandas as pd

In [57]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [58]:
## Task 1: Mark all launch sites on a map
world_map = folium.Map()
world_map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [59]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [60]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [61]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [62]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
folium.Circle(
    location=nasa_coordinate,
    radius=1000,  # in meters
    color='blue',
    fill=True,
    fill_color='blue',
    popup='NASA Johnson Space Center'
).add_to(site_map)

# Create a blue circle at NASA Johnson Space Center's coordinate with an icon showing its name
folium.Marker(
    location=nasa_coordinate,
    popup='NASA Johnson Space Center',
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# Create an icon as a text label
folium.Marker(
    location=nasa_coordinate,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12pt; color : blue;">NASA Johnson Space Center</div>'
    )
).add_to(site_map)
site_map


and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [63]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values.
# In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='#3498db',  # Light blue color
        fill=True
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    
    # Create a marker for the launch site
    folium.map.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#3498db;"><b>%s</b></div>' % row['Launch Site']  # Light blue color
        )
    ).add_to(site_map)

# Display the map
site_map



The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [64]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Define marker colors for success and failure
success_color = 'green'
failure_color = 'red'

# Iterate through the SpaceX DataFrame and add markers for each launch
for index, launch in spacex_df.iterrows():
    # Determine marker color based on launch success
    marker_color = success_color if launch['class'] == 1 else failure_color
    
    # Add marker to the map without shadows
    folium.Marker(
        location=[launch['Lat'], launch['Long']],
        icon=folium.Icon(color=marker_color, shadow=False),
        popup=launch['Launch Site']
    ).add_to(site_map)

# Display the map
site_map



Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [65]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [66]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [67]:

def get_marker_color(value):
    return 'green' if value == 1 else 'red'

# Apply the function to create the 'marker_color' column in spacex_df
spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)




*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [68]:

for index, launch in spacex_df.iterrows():
    folium.Marker(
        location=[launch['Lat'], launch['Long']],
        icon=folium.Icon(color=launch['marker_color'], shadow=False),
        popup=launch['Launch Site']
    ).add_to(marker_cluster)

marker_cluster.add_to(site_map)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [69]:
# TASK 3:from math import radians, sin, cos, sqrt, atan2
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(lat1, lon1, lat2, lon2):
   
    # Convert latitude and longitude from degrees to radians
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    # Radius of the Earth in kilometers
    R = 6371.0

    # Calculate the differences
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculate the distance
    distance = R * c

    return distance

# Launch site coordinates (KSC LC-39A)
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Iterate through each row in the DataFrame to calculate distances
for index, row in spacex_df.iterrows():
    proximity_lat = row['Lat']
    proximity_lon = row['Long']
    distance = calculate_distance(launch_site_lat, launch_site_lon, proximity_lat, proximity_lon)
    print(f"Distance between launch site and proximity {index + 1}: {distance:.2f} km")


Distance between launch site and proximity 1: 6.90 km
Distance between launch site and proximity 2: 6.90 km
Distance between launch site and proximity 3: 6.90 km
Distance between launch site and proximity 4: 6.90 km
Distance between launch site and proximity 5: 6.90 km
Distance between launch site and proximity 6: 6.90 km
Distance between launch site and proximity 7: 6.90 km
Distance between launch site and proximity 8: 6.90 km
Distance between launch site and proximity 9: 6.90 km
Distance between launch site and proximity 10: 6.90 km
Distance between launch site and proximity 11: 6.90 km
Distance between launch site and proximity 12: 6.90 km
Distance between launch site and proximity 13: 6.90 km
Distance between launch site and proximity 14: 6.90 km
Distance between launch site and proximity 15: 6.90 km
Distance between launch site and proximity 16: 6.90 km
Distance between launch site and proximity 17: 6.90 km
Distance between launch site and proximity 18: 6.90 km
Distance between la

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [70]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [71]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [72]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
coastline_lat = 28.56287
coastline_lon = -80.56792
launch_site_lat = 28.573255
launch_site_lon = -80.646895
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [79]:
# Create a Marker for the closest coastline point
closest_coastline_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup='Closest Coastline Point',
    icon=folium.Icon(color='blue')
)

# Add the closest coastline marker to the map
closest_coastline_marker.add_to(site_map)

# Display the distance between the coastline point and launch site using the icon property
distance_popup = f"Distance to Closest Coastline: {distance_coastline:.2f} km"
distance_marker = folium.Marker(
    location=[(coastline_lat + launch_site_lat) / 2, (coastline_lon + launch_site_lon) / 2],
    popup=distance_popup,
    icon=folium.Icon(color='green', icon='info-sign')
)

# Add the distance marker to the map
distance_marker.add_to(site_map)


# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [77]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
coastline_coordinates = [
     [28.50464,-80,59858],
    [28,5755,-80.58544],
    [28.38566,-80.6036]
    
    
    # Add more coastline coordinates as needed
]

# Create a map object
site_map = folium.Map(location=[28.56321, -80.57682], zoom_start=10)

# Create a Marker for the launch site
launch_site_marker = folium.Marker(location=[launch_site_lat, launch_site_lon],
                                   popup='Launch Site (KSC LC-39A)',
                                   icon=folium.Icon(color='blue', icon='rocket')).add_to(site_map)

# Create a PolyLine object using coastline coordinates and launch site coordinate
polyline = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon]] + coastline_coordinates,
                           weight=1)
site_map.add_child(polyline)
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site



In [83]:


# Define launch site coordinates
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Coastline coordinates
coastline_coordinates = [
    [28.50464, -80.59858],
    [28.5755, -80.58544],
    [28.38566, -80.6036]
    # Add more coastline coordinates as needed
]

# Create a map object
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Create a Marker for the launch site
launch_site_marker = folium.Marker(location=[launch_site_lat, launch_site_lon],
                                   popup='Launch Site (KSC LC-39A)',
                                   icon=folium.Icon(color='blue', icon='rocket')).add_to(site_map)

# Add markers for coastline coordinates
for coord in coastline_coordinates:
    folium.Marker(location=[coord[0], coord[1]], popup='Coastline Point').add_to(site_map)

# Create polylines from launch site to each coastline point
for coord in coastline_coordinates:
    polyline = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coord[0], coord[1]]], weight=1)
    site_map.add_child(polyline)

# Display the map
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
